In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
reload(fd)
reload(sd)
reload(di)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = xgb.XGBClassifier()
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)

# result 
#train f1 score: 0.1205,mcc score: 0.2288,roc auc score: 0.5325
#test f1 score: 0.0258,mcc score: 0.0633, roc auc score: 0.5067

# Scaling choice
with xgboost the scaling has no influence ?

In [ ]:
%%script false
# scaling choice

import matplotlib.pyplot as plt
import numpy as np

predictors = fd.getPredictors(dfTrx)
parameters={}
#parameters['max_iter']=1000

modelClf = xgb.XGBClassifier()
duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors,[], parameters, scaler=None)
scalers = fd.getScalers()
for key in scalers:
    print(key)
    scaler=scalers.get(key)
    modelClf = xgb.XGBClassifier()
    duration_in_s,f1Train,f1Test, scaler = fd.processModel(modelClf,dfTrx, predictors, [],parameters,scaler=scaler)

# scaler has no adding value for the result

# Hyperparameters choice
starting point
n_estimators=50,learning_rate=1

In [ ]:
%%script false

modelClf = xgb.XGBClassifier()
# starting point n_estimators=100,  max_depth=7 
dic_param={
    'n_estimators':[50,100,200],
    'max_depth':[3,5,7],
    'learning_rate':[0.01,0.05,0.1],
    'subsample':[0.8,1.0],
    'colsample_bytree':[0.8,1.0]
}
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)
# result (22/01/2025)
# result {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 1.0} scoref1 0.013

In [ ]:
%%script false
modelClf = xgb.XGBClassifier()

dic_param={
    'n_estimators':[400,500,600],
    'max_depth':[5,6,7],
    'learning_rate':[0.4,0.5,0.6],
    'subsample':[0.8,1.0],
    'colsa,.3mple_bytree':[0.8,1.0]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

# 23/01/2025
#dic 'n_estimators':[100,200,300],'max_depth':[2,3,4],'learning_rate':[0.01,0.1,0.2],'subsample':[0.8,1.0],'colsample_bytree':[0.8,1.0]
# {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 300, 'subsample': 0.8} scoref1 0.0829
#dic 'n_estimators':[200,300,400],'max_depth':[3,4,5],'learning_rate':[0.1,0.2,0.3],'subsample':[0.8,1.0],'colsample_bytree':[0.8,1.0]
# {'colsample_bytree': 1.0, 'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8} scoref1 0.1424
# dic_param={'n_estimators':[200,300,400],'max_depth':[4,5,6],'learning_rate':[0.2,0.3,0.4],'subsample':[0.8,1.0],'colsample_bytree':[0.8,1.0]
#{'colsample_bytree': 0.8, 'learning_rate': 0.4, 'max_depth': 6, 'n_estimators': 300, 'subsample': 0.8} scoref1 0.1964
#dic_param={'n_estimators':[200,300,400],'max_depth':[4,5,6],'learning_rate':[0.3,0.4,0.5],'subsample':[0.8,1.0],'colsample_bytree':[0.8,1.0]
#{'colsample_bytree': 1.0, 'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 400, 'subsample': 0.8}sco ref1 0.2498
# dic_param={'n_estimators':[300,400,500],'max_depth':[5,6,7],'learning_rate':[0.4,0.5,0.6],'subsample':[0.8,1.0],'colsample_bytree':[0.8,1.0]
#{'colsample_bytree': 1.0, 'learning_rate': 0.5, 'max_depth': 6, 'n_estimators': 500, 'subsample': 0.8} scoref1 0.2767



In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)
parameters= {'colsample_bytree': 1.0, 'learning_rate': 0.4, 'max_depth': 6, 'n_estimators': 500, 'subsample': 0.8}

modelClf = xgb.XGBClassifier()
modelClf.set_params(**parameters)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)

## result  (23/01/2025)
#test f1 score: 0.0717, mcc score: 0.1136, roc auc score: 0.5201

In [ ]:
files = fd.getAllFiles()

range = []
results = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    result= fd.print_scores(dfTrx['Class'], preds,'f1', True)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    results.append(result)

fd.plt_train_test(range, results)

#test result (22/01/2025)
# d1 f1= 0.1293
# d2 f1= 0.0169
# d3 f1= 0.0270
# d7 f1= 0.5730